# Python과 SQL 연동
* pip install PyMySQL
* cursor 객체 : 레코드를 순차적으로 가져온다. 그래서 반복문으로 변수에 저장해야한다.
    - fetchall()    : fetch는 데이터를 가져오는 기능. 모든 레코드를 다 가져온다.
    - fetchmany()   : 레코드 중 일부를 가져온다.
    - fetchone()    : 레코드를 하나씩 가져온다.
    - 객체 생략시엔 fetchall이 default값.
    - dictcursor    : 필드명을 인덱스 대신 키로 사용
* 파이썬으로 데이터 불러오거나, 반복적으로 DB에 데이터 입력해야할 때 sql보다 편하게 쓸 수 있다.
* 다만 안정성 측면에서 불안하니, 정말 필요한 거 아니면 IDE나 터미널로 db에 접근하는 것이 좋다.


In [ ]:
import pymysql
import pandas as pd

In [ ]:
config = {"host" : "127.0.0.1", "user" : "root", "passwd" : "Wpdntm123!@#", "database" : "testdb"}

In [ ]:
conn = pymysql.connect(**config)
print(conn)
conn.close()

In [ ]:
# SELECT
conn = pymysql.connect(**config)
sql = "SELECT * FROM emp limit 5" # 명령어를 담은 변수를 DB에 보내면 이 변수에 담긴 명령어를 실행하는 방식
# 명령어 변수를 실어나르고 DB에 실행시키고 결과를 파이썬으로 불러오는 객체 존재.

cursor = conn.cursor()
cursor.execute(sql)

# print(cursor)
# -> <pymysql.cursors.Cursor object at 0x7fc04148f580> 여기에 결과물이 담겨있다.

# for data in cursor:
#     print(data)

# for a, b, c, d, e, f, g, h in cursor:
#      print(a)
#      print(b,c,d,e,f,g,h)

# (a,b,c,d,e) = cursor
# print(a)

# print('='*50)

result = []
for data in cursor:
    result.append(data)

conn.close()

print(result)

df = pd.DataFrame(result, columns=['empno', 'name', 'job', 'manager', 'hiredate','salary', 'bonus', 'deptno'])
df

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)]


,empno,name,job,manager,hiredate,salary,bonus,deptno
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30


In [ ]:
# 2. Insert

conn = pymysql.connect(**config)

sql = """create table if not exists tbltest(
    empno int,
    ename varchar(10),
    sal int,
    job varchar(10)
)"""

cur = conn.cursor()
cur.execute(sql)

print(cur)

sql_insert = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"

cur.execute(sql_insert)
conn.commit()

conn.close()


# MariaDB [testdb]> SELECT * FROM tbltest;
# +-------+-------+------+------+
# | empno | ename | sal  | job  |
# +-------+-------+------+------+
# |     1 | scott | 5000 | NULL |
# +-------+-------+------+------+

In [ ]:
empno = 2
ename = "miller"
sal = 1300

conn = pymysql.connect(**config)
cur = conn.cursor()

sql_insert1 = "insert into tbltest(empno, ename, sal) values(" + str(empno) + ", '" + ename + "', " + str(sal) + ")"
cur.execute(sql_insert1)
conn.commit()
conn.close()

# MariaDB [testdb]> SELECT * FROM tbltest;
# +-------+--------+------+------+
# | empno | ename  | sal  | job  |
# +-------+--------+------+------+
# |     1 | scott  | 5000 | NULL |
# |     2 | miller | 1300 | NULL |
# +-------+--------+------+------+

In [ ]:
empno = 3
ename = "tom"
sal = 2000

conn = pymysql.connect(**config)
cur = conn.cursor()

sql_insert2 = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))
cur.execute(sql_insert2, sql_data)
conn.commit()
conn.close()

# MariaDB [testdb]> SELECT * FROM tbltest;
# +-------+--------+------+------+
# | empno | ename  | sal  | job  |
# +-------+--------+------+------+
# |     1 | scott  | 5000 | NULL |
# |     2 | miller | 1300 | NULL |
# |     3 | tom    | 2000 | NULL |
# +-------+--------+------+------+
# 3 rows in set (0.000 sec)

In [ ]:
# 두 프로그램을 연결하는 방법이라 불안정. 그래서 정식으로 코드를 짤 땐 예외처리 방식으로 진행되도록 만들어야 한다.
# try - except

empno = 4
ename = "jerry"
sal = 2000

try:
    conn = pymysql.connect(**config)
    cur = conn.cursor()

    sql_insert2 = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))

    cur.execute(sql_insert2, sql_data)
    conn.commit()

except Exception as e:
    print("error :", e)

finally:
    conn.close()

# 에러가 발생해도 반드시 연결은 끊어져야하므로! 아니면 프로그램 무거워지고 결국 뻑난다.    
 

# MariaDB [testdb]> SELECT * FROM tbltest;
# +-------+--------+------+------+
# | empno | ename  | sal  | job  |
# +-------+--------+------+------+
# |     1 | scott  | 5000 | NULL |
# |     2 | miller | 1300 | NULL |
# |     3 | tom    | 2000 | NULL |
# |     4 | jerry  | 2000 | NULL |
# |     4 | jerry  | 2000 | NULL |
# +-------+--------+------+------+ 

In [ ]:
#3. 반복문으로 DATA 여러개 입력하기
#SQL엔 없는 기능. 이런 것 때문에 파이썬과 SQL은 상호보완적

conn = pymysql.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()

conn.close()

In [ ]:
#3. Cursor

try:
    conn = pymysql.connect(**config)
    cur = conn.cursor()
    cur.execute("select * from tbltest")

    # for i in cur.fetchall():
    #     print(i)
        # (1, 'scott', 5000, None)
        # (2, 'miller', 1300, None)
        # (3, 'tom', 2000, None)
        # (4, 'jerry', 2000, None)
        # (4, 'jerry', 2000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)
        # (5, 'jane', 10000, None)

    # for i in cur.fetchmany(5):
    #     print(i)
        # (1, 'scott', 5000, None)
        # (2, 'miller', 1300, None)
        # (3, 'tom', 2000, None)
        # (4, 'jerry', 2000, None)
        # (4, 'jerry', 2000, None)

    for i in cur.fetchone():
        print(cur.fetchone())
finally:
    conn.close()

(2, 'miller', 1300, None)
(3, 'tom', 2000, None)
(4, 'jerry', 2000, None)
(4, 'jerry', 2000, None)


In [ ]:
try:
    conn = pymysql.connect(**config)
    cur = conn.cursor()
    cur.execute("select * from tbltest")

    for i in cur.fetchall():
        print(i[0], i[1], i[2])

finally:
    conn.close()

1 scott 5000
2 miller 1300
3 tom 2000
4 jerry 2000
4 jerry 2000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000


In [ ]:
try:
    conn = pymysql.connect(**config)
    cur = conn.cursor(pymysql.cursors.DictCursor)
    cur.execute("select * from tbltest")

    for i in cur.fetchall():
        print(i['empno'], i['ename'], i['sal'])

finally:
    conn.close()

1 scott 5000
2 miller 1300
3 tom 2000
4 jerry 2000
4 jerry 2000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000


In [ ]:
#csv 파일에 있는 데이터, 파이썬을 이용해 DB에 넣기
import csv
file = csv.reader(open('./input.csv'))
# print(file)
# print(next(file)) #현재 위치한 줄을 읽어오고 다음 줄에 위치하게 만드는 명령어
# print(next(file))
# print(next(file))
# print(next(file))
# print(next(file))
# print(next(file))

next(file) #필드명은 건너뛰어야하니까.
try :
    conn = pymysql.connect(**config)
    cur = conn.cursor()

    cur.execute("delete from suppliers")
    conn.commit()
    # 반복시행 시 누적되는 것을 막아주려면?

    for i in file:
        sql = "insert into suppliers values(%s, %s, %s, %s, %s)"
        cur.execute(sql, i)
    
    conn.commit()

finally :
    conn.close()


In [ ]:
try :
    conn = pymysql.connect(**config)
    cur = conn.cursor()
    cur.execute("select * from suppliers where cost > 500000")

    # for i in cur:
    #     print(i)
    # ('A', '001-1003', '5467', 750000.0, datetime.date(2014, 1, 20))
    # ('A', '001-1004', '5467', 750000.0, datetime.date(2014, 1, 20))
    # ('C', '001-1009', '3321', 615000.0, datetime.date(2017, 2, 3))
    # ('C', '001-1010', '3321', 615000.0, datetime.date(2017, 2, 10))
    # ('C', '001-1011', '3321', 615000.0, datetime.date(2017, 2, 17))
    # ('C', '001-1012', '3321', 615000.0, datetime.date(2017, 2, 24))

    file = csv.writer(open("./output.cs", "w"), delimiter = ",")
    #csv.writer는 데이터간 구분짓는 문자를 지정할 수 있다. delimiter 이용

    header = ["Supplier_Name","Invoice_Number","Part_Number","Cost","Purchase_Date"] #여백없이 해야 필드명으로 지정된다.
    file.writerow(header)

    for i in cur:
        file.writerow(i)

finally :
    conn.close()
